# The Price is Right

Today we'll build another piece of the puzzle: a ScanningAgent that looks for promising deals by subscribing to RSS feeds.

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection

In [2]:
# Initialize and constants

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [3]:
deals = ScrapedDeal.fetch(show_progress=True)

100%|██████████| 5/5 [00:55<00:00, 11.07s/it]


In [4]:
len(deals)

49

In [5]:
deals[44].describe()

"Title: Celestron AstroMaster 70AZ LT Refractor Telescope Kit for $49 + free shipping\nDetails: You'd pay over $100 elsewhere. Buy Now at Walmart\nFeatures: \nURL: https://www.dealnews.com/Celestron-Astro-Master-70-AZ-LT-Refractor-Telescope-Kit-for-49-free-shipping/21708652.html?iref=rss-c196"

In [6]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [7]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [8]:
print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Google Pixel Buds Pro 2 Earbuds w/ Fitbit Charge 6 Health Tracker for $319 + free shipping
Details: Pick your prefered colors for the Google Pixel Buds Pro 2 Noise Canceling Earbuds and Fitbit Charge 6 Advanced Fitness and Health Tracker, and you'll see this price in cart. It's $10 less than you'd pay for these items sold separately at Amazon. Buy Now at Best Buy
Fe

In [9]:
def get_recommendations():
    completion = openai.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ],
        response_format=DealSelection
    )
    result = completion.choices[0].message.parsed
    return result

In [10]:
result = get_recommendations()

In [11]:
result

DealSelection(deals=[Deal(product_description="The Google Pixel Buds Pro 2 are designed for an immersive audio experience, featuring active noise cancellation and a custom fit for enhanced comfort. They come in your choice of colors and pair seamlessly with the Fitbit Charge 6, a versatile fitness and health tracker that offers advanced monitoring of your workouts and health metrics. This bundle delivers superior sound quality for music lovers while keeping you informed on your health stats. It's perfect for those seeking both entertainment and fitness tracking in one stylish package.", price=319.0, url='https://www.dealnews.com/Google-Pixel-Buds-Pro-2-Earbuds-w-Fitbit-Charge-6-Health-Tracker-for-319-free-shipping/21708692.html?iref=rss-c142'), Deal(product_description='The Dell Inspiron 15 laptop boasts an AMD Ryzen 5 7520U quad-core CPU, ensuring smooth multitasking and speedy performance. With 8GB RAM and a 512GB SSD, this laptop provides ample space for storage and quick boot times

In [12]:
len(result.deals)

5

In [13]:
result.deals[1]

Deal(product_description='The Dell Inspiron 15 laptop boasts an AMD Ryzen 5 7520U quad-core CPU, ensuring smooth multitasking and speedy performance. With 8GB RAM and a 512GB SSD, this laptop provides ample space for storage and quick boot times. The 15.6-inch display features a full HD resolution of 1920x1080, providing vivid colors and clarity for both work and entertainment. Running on Windows 11 Home, it comes equipped for modern computing needs and is ideal for students and professionals alike.', price=330.0, url='https://www.dealnews.com/products/Dell/Dell-Inspiron-15-Ryzen-5-15-6-Laptop/373520.html?iref=rss-c39')

In [14]:
from agents.scanner_agent import ScannerAgent

In [15]:
agent = ScannerAgent()
result = agent.scan()

In [16]:
result

DealSelection(deals=[Deal(product_description='The Google Pixel Buds Pro 2 are premium wireless earbuds featuring noise-canceling technology and a variety of customizable colors. They offer high-quality audio with a comfortable fit for all-day use. The accompanying Fitbit Charge 6 tracker provides advanced health monitoring and fitness tracking capabilities, making this bundle ideal for tech-savvy users looking to enhance their daily fitness routine.', price=319.0, url='https://www.dealnews.com/Google-Pixel-Buds-Pro-2-Earbuds-w-Fitbit-Charge-6-Health-Tracker-for-319-free-shipping/21708692.html?iref=rss-c142'), Deal(product_description='The Samsung Galaxy S24 Ultra smartphone represents cutting-edge technology in mobile devices with a stunning display and high-performance capabilities. Its robust camera system excels in low-light conditions and supports high-resolution video recording. With options for 256GB and 512GB storage, this phone is designed for both everyday users and tech enth